<a href="https://colab.research.google.com/github/aayushvashistha/datalakeProject/blob/main/countries_ip_dataClean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [112]:
from pyspark.sql.functions import col, lit, when
df = spark.read.csv("challenge.csv", header= True, inferSchema= True)

In [47]:
df = df.na.drop()
df = df.dropDuplicates()
df.show()

+---------------+------------------+-----------------+----------+
|     ip_address|           Country|      Domain Name|Bytes_used|
+---------------+------------------+-----------------+----------+
|   246.97.176.4|Dominican Republic|      spotify.com|       541|
|  227.24.96.107|       Philippines|           gov.uk|       429|
| 35.104.255.113|              Laos|          nba.com|       707|
|  163.229.14.95|       Philippines|   prnewswire.com|       846|
| 155.191.132.94|             Syria|       dion.ne.jp|       610|
|  22.48.127.120|         Indonesia|elegantthemes.com|       298|
|  241.69.89.147|         Indonesia|       uol.com.br|       741|
|227.187.247.203|            Greece|         about.me|       620|
|   33.8.169.147|            Sweden|     marriott.com|       935|
|   33.194.7.236|            Malawi|       usnews.com|        84|
| 17.248.153.253|           Ukraine|   eventbrite.com|       748|
|  34.145.145.62|             China|   cloudflare.com|       540|
|253.251.2

In [48]:
df.printSchema()

root
 |-- ip_address: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Domain Name: string (nullable = true)
 |-- Bytes_used: integer (nullable = true)



In [113]:
import pyspark.sql.functions as sqlfunc
df1 = df.withColumn("is country mexico", when(col("Country") == "Mexico", "yes").otherwise("no"))
df1.show()

+---------------+--------------+-----------------+----------+-----------------+
|     ip_address|       Country|      Domain Name|Bytes_used|is country mexico|
+---------------+--------------+-----------------+----------+-----------------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|               no|
| 119.239.207.13|         China|         youtu.be|        51|               no|
|  68.69.217.210|         China|        adobe.com|        10|               no|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|               no|
|   211.13.10.68|     Indonesia|          hud.gov|        29|               no|
|   239.80.21.97|      Suriname|       smh.com.au|       218|               no|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|               no|
| 127.242.24.138|         China| surveymonkey.com|       123|               no|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|               no|
|   237.54.11.63|         China|       a

In [114]:
df2 = df1.groupBy("is country mexico").agg(sqlfunc.sum("Bytes_used").alias("total bytes used"))
df2.show()

+-----------------+----------------+
|is country mexico|total bytes used|
+-----------------+----------------+
|               no|          508076|
|              yes|            6293|
+-----------------+----------------+



In [121]:
df3 = df1.groupBy("Country").agg(sqlfunc.countDistinct(df1.ip_address).alias("number_of_ips"))
df3.sort(col("number_of_ips").desc()).show()

+--------------+-------------+
|       Country|number_of_ips|
+--------------+-------------+
|         China|          172|
|     Indonesia|          114|
|   Philippines|           65|
|        Russia|           56|
|        Brazil|           35|
|        Poland|           31|
|        Sweden|           28|
|         Japan|           25|
|Czech Republic|           23|
|      Portugal|           23|
|        France|           21|
|          Peru|           19|
|      Colombia|           17|
| United States|           15|
|     Argentina|           14|
|       Ukraine|           14|
|        Mexico|           13|
|      Thailand|           12|
|        Canada|           11|
|       Nigeria|           11|
+--------------+-------------+
only showing top 20 rows



In [96]:
df1.registerTempTable("challenge")

In [110]:
query = spark.sql("SELECT Country, COUNT(DISTINCT ip_address) AS distinct_ip_address FROM challenge GROUP BY Country")
query.show()
# df3 = query.groupBy("Country"))

+-----------+-------------------+
|    Country|distinct_ip_address|
+-----------+-------------------+
|       Chad|                  1|
|     Russia|                 56|
|   Paraguay|                  1|
|      Yemen|                  1|
|     Sweden|                 28|
|Philippines|                 65|
|   Malaysia|                  5|
|     Turkey|                  1|
|     Malawi|                  2|
|    Germany|                  5|
|    Comoros|                  1|
|Afghanistan|                  5|
|     Rwanda|                  1|
|      Sudan|                  1|
|     France|                 21|
|     Greece|                  8|
|  Sri Lanka|                  3|
|   Dominica|                  1|
|  Argentina|                 14|
|    Belgium|                  1|
+-----------+-------------------+
only showing top 20 rows

